In [1]:
import ipaddress

ip_input = input("Enter an IP address: ")

try:
    ip = ipaddress.ip_address(ip_input)
    print(f"{ip} is a valid IP address.")
    if ip.is_private:
        print("It is a private IP address.")
    else:
        print("It is a public IP address.")
except ValueError:
    print("Invalid IP address!")

Enter an IP address:  192.168.1.5


192.168.1.5 is a valid IP address.
It is a private IP address.


In [2]:
network_input = input("Enter network with CIDR (e.g., 192.168.1.0/24): ")

try:
    network = ipaddress.ip_network(network_input, strict=False)
    print(f"Network: {network}")
    print(f"Broadcast address: {network.broadcast_address}")
    hosts = list(network.hosts())
    print(f"First usable host: {hosts[0]}")
    print(f"Last usable host: {hosts[-1]}")
    print(f"Number of usable hosts: {len(hosts)}")
except ValueError:
    print("Invalid network!")

Enter network with CIDR (e.g., 192.168.1.0/24):  192.168.1.0/24


Network: 192.168.1.0/24
Broadcast address: 192.168.1.255
First usable host: 192.168.1.1
Last usable host: 192.168.1.254
Number of usable hosts: 254


In [3]:
network = ipaddress.ip_network("172.16.0.0/16")
subnets_24 = list(network.subnets(new_prefix=24))
print(f"Number of /24 subnets: {len(subnets_24)}")

Number of /24 subnets: 256


In [4]:
import socket

hostname = socket.gethostname()
local_ip = socket.gethostbyname(hostname)
ip = ipaddress.ip_address(local_ip)

print(f"Your hostname: {hostname}")
print(f"Your IP: {local_ip}")
print(f"Private? {'Yes' if ip.is_private else 'No'}")

Your hostname: LAPTOP-L0L30K6T
Your IP: 192.168.0.23
Private? Yes


In [5]:
website = "www.gold.ac.uk"
ip_address = socket.gethostbyname(website)
ip = ipaddress.ip_address(ip_address)

print(f"{website} IP address: {ip}")
print(f"Private? {'Yes' if ip.is_private else 'No'}")

www.gold.ac.uk IP address: 159.100.136.66
Private? No


In [6]:
network = ipaddress.ip_network("172.16.0.0/16")

departments = {
    "Engineering": 30,
    "Marketing": 15,
    "Finance": 10,
    "HR": 5
}

current_address = int(network.network_address)

for dept, hosts in departments.items():
    # Calculate minimum prefix to fit hosts
    bits_needed = (hosts + 2).bit_length()
    prefix = 32 - bits_needed
    subnet = ipaddress.ip_network((current_address, prefix), strict=False)
    print(f"{dept} subnet: {subnet} - usable hosts: {len(list(subnet.hosts()))}")
    current_address += 2 ** (32 - prefix)

Engineering subnet: 172.16.0.0/26 - usable hosts: 62
Marketing subnet: 172.16.0.64/27 - usable hosts: 30
Finance subnet: 172.16.0.96/28 - usable hosts: 14
HR subnet: 172.16.0.112/29 - usable hosts: 6


In [7]:
class NATTable:
    def __init__(self, public_ip):
        self.public_ip = public_ip
        self.table = {}  # (private_ip, private_port) -> public_port
        self.next_port = 10000

    def outbound(self, private_ip, private_port, dest_ip, dest_port):
        key = (private_ip, private_port)
        if key not in self.table:
            self.table[key] = self.next_port
            self.next_port += 1
        pub_port = self.table[key]
        print(f'NAT: {private_ip}:{private_port} -> {self.public_ip}:{pub_port} -> {dest_ip}:{dest_port}')
        return self.public_ip, pub_port

    def inbound(self, public_port, src_ip, src_port):
        for key, port in self.table.items():
            if port == public_port:
                print(f'Return: {src_ip}:{src_port} -> {key[0]}:{key[1]}')
                return key
        return None

    def show_table(self):
        print("\n--- NAT Table ---")
        for k, v in self.table.items():
            print(f"{k[0]}:{k[1]} -> {self.public_ip}:{v}")
        print("----------------\n")

# Test with 5 devices
nat = NATTable('203.0.113.1')
devices = ['192.168.1.10','192.168.1.11','192.168.1.12','192.168.1.13','192.168.1.14']
for i, dev in enumerate(devices):
    nat.outbound(dev, 5000+i, '93.184.216.34', 80)

nat.show_table()

NAT: 192.168.1.10:5000 -> 203.0.113.1:10000 -> 93.184.216.34:80
NAT: 192.168.1.11:5001 -> 203.0.113.1:10001 -> 93.184.216.34:80
NAT: 192.168.1.12:5002 -> 203.0.113.1:10002 -> 93.184.216.34:80
NAT: 192.168.1.13:5003 -> 203.0.113.1:10003 -> 93.184.216.34:80
NAT: 192.168.1.14:5004 -> 203.0.113.1:10004 -> 93.184.216.34:80

--- NAT Table ---
192.168.1.10:5000 -> 203.0.113.1:10000
192.168.1.11:5001 -> 203.0.113.1:10001
192.168.1.12:5002 -> 203.0.113.1:10002
192.168.1.13:5003 -> 203.0.113.1:10003
192.168.1.14:5004 -> 203.0.113.1:10004
----------------



In [8]:
#🔹 Exercise 8 – NAT & P2P

#Why peer-to-peer apps struggle with NAT:

#NAT hides private IPs behind a single public IP → other peers cannot directly connect.

#UDP/TCP hole punching is used to allow two peers to connect through NAT by temporarily opening a port mapping.